[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb)

# Hello, Pinecone!

This notebook will walk through the steps to get a simple Pinecone index up and running.


## Prerequisites

In [ ]:
!pip3 install pandas

In [ ]:
import os
import lantern_pinecone as pinecone

LANTERN_DB_URL = os.environ.get('LANTERN_DB_URL') or 'postgres://postgres@localhost:5432'
pinecone.init(LANTERN_DB_URL)

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors.

In [2]:
# Giving our index a name
index_name = "hello-pinecone"

In [3]:
# Delete the index, if an index of the same name already exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

Creating a Pinecone Index.

In [4]:
import time

dimensions = 3
pinecone.create_index(name=index_name, dimension=dimensions, metric="cosine")

# wait for index to be ready before connecting
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

In [5]:
index = pinecone.Index(index_name=index_name)

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [6]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [7]:
print(list(zip(df.id, df.vector)))

[('A', [1.0, 1.0, 1.0]), ('B', [1.0, 2.0, 3.0])]


In [8]:
index.upsert(vectors=zip(df.id, df.vector))  # insert vectors

2

In [9]:
index.describe_index_stats()

{'dimensions': 3,
 'index_fullness': 1.0,
 'total_count': 2,
 'namespaces': {'': {'vector_count': 2}}}

In [10]:
index.query(
    vector=[2., 2., 2.],
    top_k=5,
    include_values=True) # returns top_k matches

{'namespace': '',
 'matches': [{'id': 'A',
   'score': 1.0,
   'values': [1.0, 1.0, 1.0],
   'metadata': None},
  {'id': 'B',
   'score': 0.92582011,
   'values': [1.0, 2.0, 3.0],
   'metadata': None}]}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [11]:
pinecone.delete_index(index_name)